In [ ]:
import os, sys
ROOT = r'C:\Users\7459985\Desktop\2025\RDDS\001_code\meow_racer\step_1_2_rl_adv'.replace('\\', '/')
sys.path.append(ROOT)
from rlkit.torch.sac.policies.gaussian_policy import TanhGaussianPolicy
## rlkit은 policy distribution을 구현하기 위해서 torch.distributions의 확률 분포를 그대로 사용하는 것이 아니라, 따로 Distribution class를 구현해 놓음.
## Beta distribution도 구현이 되어 있기 때문에 사용해 봐도 좋을 듯.


In [5]:
policy = TanhGaussianPolicy(hidden_sizes=[256,256], obs_dim=33, action_dim=2)

In [7]:
for i, fc in enumerate(policy.fcs):
    print(fc)

Linear(in_features=33, out_features=256, bias=True)
Linear(in_features=256, out_features=256, bias=True)


In [39]:
from torch.distributions import Normal as TorchNormal
from rlkit.torch.distributions import TanhNormal
from rlkit.torch.sac.policies import MakeDeterministic, TanhGaussianPolicy
import numpy as np
import torch
# dist = TorchNormal(0, 1)
# dist = TanhNormal(torch.Tensor([[0, 0]]), torch.Tensor([[1,1]]))
policy = TanhGaussianPolicy(hidden_sizes=[256,256],obs_dim=33, action_dim=2)
dummy_obs = torch.FloatTensor(np.random.random((10, 33)))
dist = policy(dummy_obs)
# log_prob = dist.log_prob(torch.FloatTensor(np.array(
#     [[[0.2], [-0.4]], [[-0.3], [0.3]]
# ]))) #하나의 batch안의 action이 [[action_1], [action_2]]이렇게 2차원이고, 이런게 여러 batch있어야 함.
# print(log_prob)
# squeezed_log_prob = log_prob.sum(dim=1, keepdim=True)
# print(squeezed_log_prob)

In [55]:
import torch
torch.save(policy.state_dict(), 'test.pth')

In [57]:
from collections import namedtuple
SACLosses = namedtuple(
    'SACLosses',
    'policy_loss qf1_loss qf2_loss alpha_loss',
)
loss = SACLosses(policy_loss=0, qf1_loss=1, qf2_loss=2, alpha_loss=3)
print(loss)

SACLosses(policy_loss=0, qf1_loss=1, qf2_loss=2, alpha_loss=3)


In [60]:
for key, value in loss._asdict().items():
    print(key, value)

policy_loss 0
qf1_loss 1
qf2_loss 2
alpha_loss 3


In [56]:
policy.state_dict()

OrderedDict([('fc0.weight',
              tensor([[-0.0322, -0.0443,  0.0458,  ..., -0.0364, -0.0493,  0.0388],
                      [ 0.0518, -0.0343,  0.0228,  ..., -0.0376, -0.0567,  0.0300],
                      [ 0.0494,  0.0034,  0.0404,  ..., -0.0308, -0.0090, -0.0350],
                      ...,
                      [-0.0249, -0.0513, -0.0461,  ..., -0.0218,  0.0164,  0.0129],
                      [-0.0073, -0.0521, -0.0072,  ..., -0.0478, -0.0159, -0.0220],
                      [-0.0272, -0.0363,  0.0571,  ..., -0.0034, -0.0587,  0.0142]])),
             ('fc0.bias',
              tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                      0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                      0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                      0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [44]:
new_obs_actions, log_pi = dist.rsample_and_logprob()

In [45]:
new_obs_actions

tensor([[ 0.7626, -0.2662],
        [-0.5262,  0.4784],
        [ 0.7537, -0.9628],
        [-0.8191,  0.4484],
        [-0.4677, -0.2465],
        [ 0.6115, -0.2867],
        [ 0.8072, -0.6073],
        [ 0.8721,  0.7336],
        [ 0.9635, -0.5448],
        [ 0.4816, -0.5443]], grad_fn=<TanhBackward0>)

In [46]:
log_pi

tensor([-1.4332, -1.5607, -0.8287, -1.2842, -1.6885, -1.5811, -1.1985, -0.9745,
        -1.0224, -1.5473], grad_fn=<AddBackward0>)

In [ ]:
eval_policy = MakeDeterministic(policy) #delta distribution
single_obs = torch.Tensor(np.random.random((1, 33)))
delta_dist = eval_policy(single_obs) #현재 observation을 기반으로 mean, std가 neural net에 의해 계산되면 이로 distribution을 build해서 return
delta_dist.mean #결국에 정규 분포를 따른다고 한다면, distribution의 평균이 deterministic한 policy라고 가정할 때 Q-function의 최대치를 기록하게 하는 action이라고 볼 수 있다.

tensor([[-0.0005,  0.0002]], grad_fn=<TanhBackward0>)

In [52]:
from rlkit.samplers.data_collector.path_collector import MdpPathCollector

In [ ]:
def _get_single_rollout(env, agent, obs, preprocess_obs_for_policy_fn=None):
    '''[TODO]
    1. 어차피 environment normalization wrapper을 사용하는데, 혹시 observation_normalization_fn이 있어야 하는지 확인 필요
    '''
    rollout_dict = {}
    
    if preprocess_obs_for_policy_fn is not None:
        preprocess_obs_for_policy_fn = lambda x: x
        
    o_for_agent = preprocess_obs_for_policy_fn(obs)
    # get distribution from observation -> samples from the distribution
    # when evaluation mode, the distribution is Delta distribution (deterministic)
    action, agent_info = agent.get_action(o_for_agent, **get_action_kwargs) # action, empty dict
    
    
    env_step_ret = env.step(copy.deepcopy(action))
    if len(env_step_ret) == 5:
        next_o, r, terminated, truncated, env_info = env_step_ret
        done = terminated | truncated
    elif len(env_step_ret) == 4:
        next_o, r, done, env_info = env_step_ret
    
    
    rollout_dict['observation'] = next_o
    rollout_dict['action'] = action
    rollout_dict['terminal'] = done
    
    return rollout_dict
    
    
def evaluate(eval_policy, 
             eval_env,
             max_path_length,
             ):
    '''evaluation code referenced from the MdpPathCollector in samplers/data_collector/path_collector.py,
    and rollout function from samplers/rollout_functions.py
    (하나의 track만을 사용해서 )'''

    path_length = 0
    eval_policy.reset()
    o, env_info_dict = eval_env.reset()
    
    observations = []
    actions = []
    rewards = []
    terminal = []
    
    while path_length < max_path_length:
        single_rollout_dict = _get_single_rollout(env=eval_env, agent=eval_policy)
        
        observations.append(rollout_dict['observation'])
        actions.append(rollout_dict['action'])
        is_done = rollout_dict['terminal']
        terminal.append(is_done)
        if is_done:
            break

    actions = np.array(actions)
    if len(actions.shape) == 1:
        actions = np.expand_dims(actions, 1)
    observations = np.array(observations)
    rewards = np.array(rewards)
    if len(rewards.shape) == 1:
        rewards = rewards = rewards.reshape(-1, 1) #보통 상수이기 때문에 reshape 사용
    
    return {
        'observations': observations,
        'rewards': rewards,
        'actions': actions,
        'terminal': terminal
    }
    
    
        